# Piezo Tracking Analysis

This notebook can be used to analyze data from C-Trap experiments in which the force-distance curve of a DNA molecule is reconstructed from trap steering sensor data, instead of video tracking.

We will assume the following configuration of the experiment:

         tethered DNA
         v
    O----------O <--> trap 1 is moved along X
    ^          ^
    trap 2     trap 1 with bead
    
So: trap 2 is positioned on the left, and is kept fixed throughout the experiments. Trap 1 is positioned to the right of trap 2, such that the tethered DNA molecule is horizontal. Stretching of the DNA is done by moving trap 1 to the right.

In the above configuration, we only have to consider distances and forces along the X (horizontal) axis. We will conveniently ignore any forces in Y.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from lumicks import pylake

## I. Sensor-to-Distance Calibration

As a first step, one should record a file with "baseline data". Here, beads are captured in traps 1 and 2 and tracked using video tracking, but no DNA molecule is tethered. One then places trap 1 first on the left of the available range of motion, and keeps it there for a second or so. Then trap 1 is moved to the far right, and kept there again for a short time.

These two data points are then used to calculate a calibration value between the video tracking, and the trap steering sensor data.

**[TODO]** For now, the code below takes the whole baseline curve, and runs a linear fit on the full sensor value vs. video tracking plot. Taking the static endpoints would be more accurate, as the trap would be standing still there.

In [ ]:
# Load data file with calibration info
cal_data = pylake.File('example.h5')

In [ ]:
# Extract video tracking distance data and sensor data
# NOTE: The distance data should already have the bead diameter subtracted
# by Bluelake.
sensor_data = cal_data['Trap position']['1X']
sensor_data_start = sensor_data.timestamps[0]
sensor_data_stop = sensor_data.timestamps[-1]
dist_data = cal_data.distance1[sensor_data_start:sensor_data_stop]

dist_data_t = dist_data.timestamps
sensor_data_t = sensor_data.timestamps
sensor_data_filter = np.isin(sensor_data_t, dist_data_t, assume_unique=True)
sensor_data_filtered = sensor_data.data[sensor_data_filter]

In [ ]:
cal = np.polyfit(sensor_data_filtered, dist_data.data, 1)
cal_fun = np.poly1d(cal)
print(f'Fit result for sensor/distance calibration: d = {cal[0]:.4f} * x + {cal[1]:.3f}')

In [ ]:
plt.figure(figsize=(8,6))
plt.title('Trap 1 sensor data vs. video tracking distance')
plt.scatter(sensor_data_filtered, dist_data.data, s=2)
plt.plot(sensor_data_filtered, cal_fun(sensor_data_filtered), '-r')
plt.xlabel('Trap 1 sensor $x$ (V)')
plt.ylabel('Video tracking distance $d$ (um)')
plt.show()

In [ ]:
del cal_data

## II. Force Baseline Calibration

If traps 1 and 2 are in very close proximity to each other (say, around 1 um or less), they will directly interact with each other. This generates a small but significant, distance-dependent force signal. This _baseline_ force needs to be subtracted from the data, in order to obtain the _actual_ force on the tether.

(Note that this is not the same as the trap-trap interference signal often observed in other optical tweezers systems: such interference is actively canceled out by the optical design of the C-Trap.)

To keep the subtraction performant, we'll fit a polynomial to the baseline curve, and use that polynomial for the subtraction later on.

In [ ]:
baseline_data = pylake.File('example2.h5')

In [ ]:
downsampling_factor = 100  # to speed up the fit
force_data = -baseline_data.force2x.downsampled_by(factor=downsampling_factor).data
sensor_data = baseline_data['Trap Position']['Trap 1x'].downsampled_by(factor=downsampling_factor).data

baseline_curve = np.poly1d(np.polyfit(sensor_data, force_data, deg=7))

In [ ]:
plt.figure(figsize=(8,5))
plt.scatter(sensor_data, force_data, s=3)
plt.plot(sensor_data, baseline_curve(sensor_data), '-r')
plt.xlabel('Trap 1 sensor $x$ (V)')
plt.ylabel('Force baseline (pN)')
plt.title('Force baseline')
plt.show()

In [ ]:
del baseline_data

## III. Data Analysis

For the actual data analysis, we can now convert trap 1 sensor positions into trap-to-trap distances. However, there are three more corrections we need to make:

1. **Subtract bead displacement.**

  At high forces, the actual configuration of the experiment looks more like this:
  
       trap 2 center
       |  bead in trap 2
       v  v
       x  O-------O  x
                  ^  ^
                  |  trap 1 center
                  bead in trap 1
                     
   Because of the forces acting on the beads, they are pulled out of the centers of the optical traps. We need to subtract this "bead displacement" from the trap-to-trap distance, in order to obtain the real tether length.
   
2. **Differential force detection.** 

  Following the method outlined in Moffitt *et al.* (2006), we can slightly enhance the resolution of our force signal. See the cited paper for more information.
   
3. **Subtract bead diameter.**

  Instead of the distance from bead center to bead center, we need the tether length. This "bead diameter subtraction" was already taken care of in step I, though, and as such, we won't need to apply this correction again.

In [ ]:
# Load data file for actual experiment
pull_data = pylake.File('example2.h5')

In [ ]:
# Extract the data that we need:
# 1. Trap 1 sensor data
# 2. Force 2x high-resolution data
# 3. Trap stiffness ("kappa", pN/nm)

def get_calibration_value(file, force_id, key):
    """Extract a calibration value from the first available calibration's metadata
    
    Args:
    - f (lumicks.pylake.File)
    - force_id (str): name of the force channel (e.g., '1x', '2y')
    - key (str): name of the calibration key
    """
    calibration_info = file['Calibration']
    first_calibration = calibration_info[list(calibration_info.h5.keys())[0]]
    return first_calibration[f'Force {force_id}'].h5.attrs[key]

def nm_to_um(x):
    return 1e-3*x

t_data = (pull_data.force2x.timestamps - pull_data.force2x.timestamps[0])/1e9

# Differential force detection: combine force signals from traps 1 and 2.
# The minus sign takes care of the different directions of both forces.
orig_force_data = (pull_data.force1x.data - pull_data.force2x.data) / 2  # in [pN]
sensor_data = pull_data['Trap Position']['Trap 1x'].data  # in [V]

# Subtract force baseline
force_data = orig_force_data - baseline_curve(sensor_data)

trap_to_trap_dist = cal_fun(sensor_data)

trap_stiffness = get_calibration_value(pull_data, '1x', 'kappa (pN/nm)')
print(f'Trap stiffness: {trap_stiffness:.2f} pN/nm')
tether_length = trap_to_trap_dist - 2*nm_to_um(force_data/trap_stiffness)

In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(tether_length, force_data, s=1)
plt.xlabel('Tether length $d$ (um)')
plt.ylabel('Force (pN)')
plt.show()

In [ ]:
plt.figure(figsize=(8,6))
plt.title('Force baseline subtraction')
plt.scatter(t_data, orig_force_data, s=2, label='Original data')
plt.scatter(t_data, force_data, s=2, label='Corrected data')
plt.xlabel('t (s)')
plt.ylabel('$F_{2x}$ (pN)')
plt.legend()
plt.show()

In [ ]:
def write_to_file(filename, headers, data_list):
    new_list = []
    for data in data_list:
        new_list.append(data.reshape((len(data),1)))
    to_write = np.concatenate(new_list, axis=1)
    np.savetxt(filename, to_write, delimiter=',', header=headers)

In [ ]:
write_to_file(filename='OutputFile2.csv', headers='length,time,Raw Force,Corrected Force',
                     data_list = [tether_length, t_data, orig_force_data, force_data])